In [1]:
import os 
import pandas as pd

In [5]:
path ="Country Weather.csv"

weather_original = pd.read_csv(path)
weather_original.head()

,Country,Latitude,Longitude,Elevation,Date,Precipitation,Temp Avg,Temp Max,Temp Min
0,China,30.6,114.05,34.0,1/22/2020,0.48,42.0,46.0,39.0
1,China,30.6,114.05,34.0,1/23/2020,0.13,45.0,NaN,39.0
2,China,30.6,114.05,34.0,1/24/2020,0.23,43.0,47.0,NaN
3,China,30.6,114.05,34.0,1/25/2020,0.18,40.0,45.0,37.0
4,China,30.6,114.05,34.0,1/26/2020,0.02,38.0,42.0,NaN


In [23]:
# Drop unneeded columns
weather_df = weather_original[['Country', 'Precipitation', 'Temp Avg', 'Temp Max', 'Temp Min']]

# Renaming Countries
# weather_df.replace('')

In [24]:
# Changing the date from days to months
date_format = pd.to_datetime(weather_original['Date'], format='%m/%d/%Y').dt.strftime('%m/%Y')

# Adding new date column to df
weather_df['Date'] = date_format
weather_df

/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Country,Precipitation,Temp Avg,Temp Max,Temp Min,Date
0,China,0.48,42.0,46.0,39.0,01/2020
1,China,0.13,45.0,NaN,39.0,01/2020
2,China,0.23,43.0,47.0,NaN,01/2020
3,China,0.18,40.0,45.0,37.0,01/2020
4,China,0.02,38.0,42.0,NaN,01/2020
...,...,...,...,...,...,...
3311,Bahrain,NaN,66.0,NaN,63.0,02/2020
3312,Bahrain,NaN,67.0,NaN,64.0,02/2020
3313,Bahrain,NaN,67.0,NaN,61.0,02/2020
3314,Bahrain,NaN,71.0,NaN,66.0,03/2020


In [38]:
# Individual dfs to hold values while calculating monthly values
avg_weather = weather_df[['Country','Date', 'Temp Avg']]
min_weather = weather_df[['Country','Date', 'Temp Min']]
max_weather = weather_df[['Country','Date', 'Temp Max']]
precip_weather = weather_df[['Country','Date', 'Precipitation']]

# Find monthly precip values and min, max and avg temps
avg_weather_df1 = avg_weather.groupby(['Country', 'Date']).mean()
min_weather_df1 = min_weather.groupby(['Country', 'Date']).min()
max_weather_df1 = max_weather.groupby(['Country', 'Date']).max()
precip_weather_df1 = precip_weather.groupby(['Country', 'Date']).sum()


# Reset index for dfs
avg_weather_df = avg_weather_df1.reset_index()
min_weather_df = min_weather_df1.reset_index()
max_weather_df = max_weather_df1.reset_index()
precip_weather_df = precip_weather_df1.reset_index()


print(avg_weather_df)
print(min_weather_df)
print(max_weather_df)
print(precip_weather_df)

           Country     Date   Temp Avg
0      Afghanistan  01/2020  42.000000
1      Afghanistan  02/2020  51.379310
2      Afghanistan  03/2020  60.000000
3          Algeria  01/2020  44.300000
4          Algeria  02/2020  49.172414
..             ...      ...        ...
239  United States  02/2020  53.962963
240  United States  03/2020  55.000000
241        Vietnam  01/2020  79.000000
242        Vietnam  02/2020  80.931034
243        Vietnam  03/2020  82.000000

[244 rows x 3 columns]
           Country     Date  Temp Min
0      Afghanistan  01/2020      31.0
1      Afghanistan  02/2020      29.0
2      Afghanistan  03/2020      46.0
3          Algeria  01/2020      31.0
4          Algeria  02/2020      26.0
..             ...      ...       ...
239  United States  02/2020      35.0
240  United States  03/2020      42.0
241        Vietnam  01/2020       NaN
242        Vietnam  02/2020      71.0
243        Vietnam  03/2020       NaN

[244 rows x 3 columns]
           Country     Date 

In [46]:
# Create df to hold all of January's data
jan_avg = avg_weather_df.loc[(avg_weather_df["Date"] == "01/2020")]
jan_min = min_weather_df.loc[(min_weather_df["Date"] == "01/2020")]
jan_max = max_weather_df.loc[(max_weather_df["Date"] == "01/2020")]
jan_precip = precip_weather_df.loc[(precip_weather_df["Date"] == "01/2020")]

# Merge back into one df
part_jan = pd.merge(jan_precip, jan_avg,  on="Country")
other_jan = pd.merge(jan_min, jan_max, on="Country")
january_data = pd.merge(part_jan, other_jan, on="Country")

# Dropping date columns and renaming other columns
january_df = january_data[['Country','Precipitation', 'Temp Avg', 'Temp Min', 'Temp Max']]
january_df = january_df.rename(columns={'Precipitation':'Jan Precipitation',
                            'Temp Avg':'Jan Temp Avg', 'Temp Min':'Jan Min Temp',
                            'Temp Max':'Jan Max Temp'})

january_df.head()

,Country,Jan Precipitation,Jan Temp Avg,Jan Min Temp,Jan Max Temp
0,Afghanistan,0.78,42.0,31.0,56.0
1,Algeria,0.09,44.3,31.0,NaN
2,Argentina,0.51,74.5,49.0,96.0
3,Armenia,0.00,17.4,-8.0,41.0
4,Australia,0.07,80.4,69.0,111.0


In [47]:
# Create df to hold all of February's data
feb_avg = avg_weather_df.loc[(avg_weather_df["Date"] == "02/2020")]
feb_min = min_weather_df.loc[(min_weather_df["Date"] == "02/2020")]
feb_max = max_weather_df.loc[(max_weather_df["Date"] == "02/2020")]
feb_precip = precip_weather_df.loc[(precip_weather_df["Date"] == "02/2020")]

# Merge back into one df
part_feb = pd.merge(feb_precip, feb_avg,  on="Country")
other_feb = pd.merge(feb_min, feb_max, on="Country")
february_data = pd.merge(part_feb, other_feb, on="Country")

# Dropping date columns and renaming other columns
february_df = february_data[['Country','Precipitation', 'Temp Avg', 'Temp Min', 'Temp Max']]
february_df = february_df.rename(columns={'Precipitation':'Feb Precipitation',
                            'Temp Avg':'Feb Temp Avg', 'Temp Min':'Feb Min Temp',
                            'Temp Max':'Feb Max Temp'})

february_df.head()

,Country,Feb Precipitation,Feb Temp Avg,Feb Min Temp,Feb Max Temp
0,Afghanistan,1.42,51.379310,29.0,79.0
1,Algeria,0.00,49.172414,26.0,NaN
2,Argentina,1.06,68.275862,41.0,92.0
3,Armenia,0.53,24.571429,-8.0,45.0
4,Australia,14.87,73.785714,64.0,100.0


In [48]:
# Create df to hold all of March's data
march_avg = avg_weather_df.loc[(avg_weather_df["Date"] == "03/2020")]
march_min = min_weather_df.loc[(min_weather_df["Date"] == "03/2020")]
march_max = max_weather_df.loc[(max_weather_df["Date"] == "03/2020")]
march_precip = precip_weather_df.loc[(precip_weather_df["Date"] == "03/2020")]

# Merge back into one df
part_march = pd.merge(march_precip, march_avg,  on="Country")
other_march = pd.merge(march_min, march_max, on="Country")
march_data = pd.merge(part_march, other_march, on="Country")

# Dropping date columns and renaming other columns
march_df = march_data[['Country','Precipitation', 'Temp Avg', 'Temp Min', 'Temp Max']]
march_df = march_df.rename(columns={'Precipitation':'March Precipitation',
                            'Temp Avg':'March Temp Avg', 'Temp Min':'March Min Temp',
                            'Temp Max':'March Max Temp'})

march_df.head()

,Country,March Precipitation,March Temp Avg,March Min Temp,March Max Temp
0,Afghanistan,0.00,60.0,46.0,75.0
1,Algeria,0.00,52.0,42.0,NaN
2,Argentina,0.01,74.0,56.0,89.0
3,Armenia,0.16,33.5,28.0,40.0
4,Australia,0.00,77.5,67.0,103.0


In [53]:
# Merging monthly dfs into one
partial_df = pd.merge(january_df, february_df, on="Country")
covid_weather_df = pd.merge(partial_df, march_df, on="Country")

# Dropping unneed columns
covid_weather_df.head()

,Country,Jan Precipitation,Jan Temp Avg,Jan Min Temp,Jan Max Temp,Feb Precipitation,Feb Temp Avg,Feb Min Temp,Feb Max Temp,March Precipitation,March Temp Avg,March Min Temp,March Max Temp
0,Afghanistan,0.78,42.0,31.0,56.0,1.42,51.379310,29.0,79.0,0.00,60.0,46.0,75.0
1,Algeria,0.09,44.3,31.0,NaN,0.00,49.172414,26.0,NaN,0.00,52.0,42.0,NaN
2,Argentina,0.51,74.5,49.0,96.0,1.06,68.275862,41.0,92.0,0.01,74.0,56.0,89.0
3,Armenia,0.00,17.4,-8.0,41.0,0.53,24.571429,-8.0,45.0,0.16,33.5,28.0,40.0
4,Australia,0.07,80.4,69.0,111.0,14.87,73.785714,64.0,100.0,0.00,77.5,67.0,103.0


In [54]:
# Save df to a csv file
covid_weather_df.to_csv ('weather_cleaned', index = False, header = True)

In [52]:
country_list = covid_weather_data['Country'].tolist()
country_list

['Afghanistan',
 'Algeria',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Belarus',
 'Belgium',
 'Bosnia',
 'Brazil',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Chile',
 'China',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'Estonia',
 'Faroe Islands',
 'Finland',
 'France',
 'Georgia',
 'Germany',
 'Greece',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Jordan',
 'Kuwait',
 'Latvia',
 'Lebanon',
 'Lithuania',
 'Malaysia',
 'Mexico',
 'Morocco',
 'Netherlands',
 'Nigeria',
 'Norway',
 'Oman',
 'Pakistan',
 'Peru',
 'Philippines',
 'Poland',
 'Portugal',
 'Qatar',
 'Romania',
 'Russia',
 'Saudi Arabia',
 'Senegal',
 'Serbia',
 'Singapore',
 'Slovakia',
 'Slovenia',
 'South Africa',
 'South Korea',
 'Spain',
 'Sri Lanka',
 'Switzerland',
 'Thailand',
 'Togo',
 'Tunisia',
 'Ukraine',
 'United Arab Emirates',
 'United Kingdo

In [56]:
len(country_list)

81